In [96]:
import pandas as pd
import numpy as np
import math

# I. Numereisation

## 1. Recuperation de donnee

In [97]:
def load_data_from_csv(csv_path:str):
    data = pd.read_csv(csv_path)
    return data

## 2. Encodage

In [98]:
def encodage(data,indexes:list,encode_rules):
    ''' Permet de faire l'encodage des donnees 
        ARGS
            - data : la liste des donnees
            - indexes : la listes des colonnes a encoder
            - encode_rules : un dictionaire des regles a suivre pour encodes la valeur des donnees
    '''
    for i in range(len(data)):
        for index in indexes:
            value = data.loc[i,index]
            try:
                value = encode_rules[index][value]
            except :
                value = encode_rules[value]
            data.loc[i,index] = value

# II. Entropie

In [99]:
def entopie_piece(repetion:int,total:int):
    result = repetion / total
    return result

In [100]:
def entropie(data:pd.DataFrame,class_attribute:str):
    ''' Etablir l'entropie a partir d'une colonne de classe 
        PARAMETERS :
            - data : les donnees pour etablir l'entropie
            - class_attribute : Le nom de la colonne qui represente les donnee pour l'entropie
        RETURN :
            - entropie , la valeur entropique des donnees par rapport au colonne donnee
    '''
    # Le nombre total de donnee
    data_lenght = len(data)
    # Recuperer les valeurs de la classe
    class_values = data[class_attribute]
    # Recupere en unique les valeur de la class
    class_unique_values = set(class_values)
    # Calculer la repetions de chacunes des valeurs parmi les donnees
    repetitions = {
        valeur: class_values.tolist().count(valeur) 
        for valeur in class_unique_values
    }
    # Le resultat attendue
    entropie = 0
    # Calcul de l'entropie
    for value in class_unique_values:
        # La piece entropique de la valeur
        pc = entopie_piece(repetitions[value],data_lenght)
        # Incrementation de la somme
        it = -(pc) * math.log2(pc)
        entropie += it
    return entropie

# III. Meilleure Caracteristique

## 1. Gain

### 1.1. Separation de donnee

In [101]:
def split_data(dataframe, colonne):
    """
    Sépare les données d'un DataFrame selon une colonne spécifique en regroupant les mêmes critères.

    Parameters:
    dataframe (pd.DataFrame): Le DataFrame contenant les données.
    colonne (str): Le nom de la colonne selon laquelle séparer les données.

    Returns:
    dict: Un dictionnaire où les clés sont les valeurs uniques de la colonne et les valeurs sont les DataFrames correspondants.
    """
    groupes = dataframe.groupby(colonne)
    resultats = {nom: groupe for nom , groupe in groupes}
    return resultats

### 1.2. Fonction de gain

In [102]:
def gain(data:pd.DataFrame,ref_attribute:str,class_attribute:str):
    # Entropie de l'ensemble du data
    entropie_S = entropie(data,class_attribute)
    # Le gain attendue
    gain = entropie_S
    # Le nombre de donnee total
    nS = len(data)
    # Separation des donnees en plusieurs sous section de donnee
    data_sv = split_data(data,ref_attribute)
    for index, data_part in data_sv.items():
        # Le nombre de sous section
        nSv = len(data_part)
        sv = nSv/nS
        # Entropie du sous section
        entropie_SV = entropie(data_part,class_attribute)

        gain -= sv*entropie_SV
    return gain

## 2. Calculs des GAINS

In [103]:
def gains(data:pd.DataFrame,class_attribute:str):
    # Recuperation des criteres possibles
    indexes = data.columns
    print (indexes)

# Main

In [104]:
# Recuperation des donnees
data = load_data_from_csv("data/spam.csv")

# Encodage des donnees
encodage(
    data,
    indexes=['Previsions','Temperature','Humidite','Vent','Classe'],
    encode_rules={
        'Ensoleille':1,'Nuageux':2,'Pluvieux':3,
        'Chaud':3,'Moyen':2,'Frais':1,
        'Elevee':2,'Normale':1,
        'Faible':1,'Fort':2,
        'Oui':0,'Non':1
    }
)
# Etablir l'entropie
gains(data,'Classe')

Index(['Jour', 'Previsions', 'Temperature', 'Humidite', 'Vent', 'Classe'], dtype='object')
